In [2]:
import numpy as np
import pandas as pd
data = pd.read_csv(r"D:\Github portfolio\Hemophilia\Hemophilia-Severity-Predictor\datasets\resampled_data_adasyn.csv")
data.head()

,cDNA,Amino acid (HGVS),Amino acid (Legacy),pos,Domain_A2,Domain_A3,Domain_B,Domain_C1,Domain_C2,Domain_Signal Peptide,...,Locationingene_7,Locationingene_8,Locationingene_9,nitroBaseBef_C,nitroBaseBef_G,nitroBaseBef_T,nitroBaseAft_C,nitroBaseAft_G,nitroBaseAft_T,Severity
0,-1.411869,-1.412448,-1.412429,2.224128,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
1,-1.499705,-1.499467,-1.500672,-0.958553,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2
2,-1.499705,-1.499467,-1.500672,-0.958553,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2
3,-1.499705,-1.499467,-1.500672,-0.958553,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2
4,-1.499705,-1.499467,-1.500672,-0.958553,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2


In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K

# Load your data
df = pd.read_csv(r"D:\Github portfolio\Hemophilia\Hemophilia-Severity-Predictor\datasets\resampled_data_adasyn.csv")

# Separate features and labels
X = df.drop(columns=['Severity'])
y = df['Severity']

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert labels to one-hot encoding
y_one_hot = to_categorical(y_encoded)

# K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # Adjust n_splits as needed

fold_accuracies = []  # List to store accuracy of each fold

fold = 1
for train_index, val_index in kf.split(X_scaled):
    print(f"Processing fold {fold}")
    
    # Split data
    X_train, X_val = X_scaled[train_index], X_scaled[val_index]
    y_train, y_val = y_one_hot[train_index], y_one_hot[val_index]
    
    # Reshape for RNN
    X_train_rnn = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_val_rnn = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
    
    # Define the RNN model
    model = Sequential()
    model.add(SimpleRNN(units=50, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), return_sequences=False))
    model.add(Dense(3, activation='softmax'))  # Assuming Severity has 3 classes: 0, 1, 2

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model
    model.fit(X_train_rnn, y_train, epochs=10, batch_size=32, validation_data=(X_val_rnn, y_val), verbose=1)
    
    # Evaluate the model
    loss, accuracy = model.evaluate(X_val_rnn, y_val)
    print(f'Fold {fold} - Loss: {loss}, Accuracy: {accuracy}')
    
    # Store the accuracy of this fold
    fold_accuracies.append(accuracy)
    
    # Increment fold counter
    fold += 1

    # Clear the model from memory
    K.clear_session()

# Calculate average accuracy
average_accuracy = np.mean(fold_accuracies)
print(f'Average Accuracy across all folds: {average_accuracy}')


Processing fold 1
Epoch 1/10
216/216 [==============================] - 2s 4ms/step - loss: 0.9397 - accuracy: 0.5659 - val_loss: 0.8311 - val_accuracy: 0.6175
Epoch 2/10
216/216 [==============================] - 1s 3ms/step - loss: 0.7588 - accuracy: 0.6533 - val_loss: 0.8064 - val_accuracy: 0.6279
Epoch 3/10
216/216 [==============================] - 1s 3ms/step - loss: 0.6997 - accuracy: 0.6905 - val_loss: 0.7956 - val_accuracy: 0.6528
Epoch 4/10
216/216 [==============================] - 1s 3ms/step - loss: 0.6618 - accuracy: 0.7067 - val_loss: 0.7861 - val_accuracy: 0.6678
Epoch 5/10
216/216 [==============================] - 1s 3ms/step - loss: 0.6311 - accuracy: 0.7258 - val_loss: 0.7696 - val_accuracy: 0.6661
Epoch 6/10
216/216 [==============================] - 1s 3ms/step - loss: 0.6049 - accuracy: 0.7376 - val_loss: 0.7764 - val_accuracy: 0.6817
Epoch 7/10
216/216 [==============================] - 1s 3ms/step - loss: 0.5836 - accuracy: 0.7409 - val_loss: 0.7775 - val_accur

In [7]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam

# Load your data
df = pd.read_csv(r"D:\Github portfolio\Hemophilia\Hemophilia-Severity-Predictor\datasets\resampled_data_adasyn.csv")

# Separate features and labels
X = df.drop(columns=['Severity'])
y = df['Severity']

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert labels to one-hot encoding
y_one_hot = to_categorical(y_encoded)

# K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # Adjust n_splits as needed

fold_accuracies = []  # List to store accuracy of each fold

fold = 1
for train_index, val_index in kf.split(X_scaled):
    print(f"Processing fold {fold}")
    
    # Split data
    X_train, X_val = X_scaled[train_index], X_scaled[val_index]
    y_train, y_val = y_one_hot[train_index], y_one_hot[val_index]
    
    # Reshape for RNN
    X_train_rnn = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_val_rnn = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
    
    # Define the RNN model
    model = Sequential()
    model.add(SimpleRNN(units=100, return_sequences=True, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])))
    model.add(Dropout(0.5))
    model.add(SimpleRNN(units=100, return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))  # Assuming Severity has 3 classes: 0, 1, 2

    # Compile the model
    optimizer = Adam(learning_rate=0.01)  # You can experiment with different learning rates
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=3)
    model_checkpoint = ModelCheckpoint(f'best_model_fold_{fold}.h5', save_best_only=True)

    # Train the model
    model.fit(X_train_rnn, y_train, epochs=20, batch_size=32, validation_data=(X_val_rnn, y_val),
              callbacks=[early_stopping, model_checkpoint], verbose=1)
    
    # Evaluate the model
    loss, accuracy = model.evaluate(X_val_rnn, y_val)
    print(f'Fold {fold} - Loss: {loss}, Accuracy: {accuracy}')
    
    # Store the accuracy of this fold
    fold_accuracies.append(accuracy)
    
    # Increment fold counter
    fold += 1

    # Clear the model from memory
    K.clear_session()

# Calculate average accuracy
average_accuracy = np.mean(fold_accuracies)
print(f'Average Accuracy across all folds: {average_accuracy}')


Processing fold 1
Epoch 1/20
216/216 [==============================] - 5s 10ms/step - loss: 1.0225 - accuracy: 0.5232 - val_loss: 0.8384 - val_accuracy: 0.5961
Epoch 2/20
 29/216 [===>..........................] - ETA: 0s - loss: 0.8753 - accuracy: 0.5991

C:\Users\saray\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


216/216 [==============================] - 1s 6ms/step - loss: 0.8919 - accuracy: 0.5869 - val_loss: 0.8267 - val_accuracy: 0.6123
Epoch 3/20
216/216 [==============================] - 1s 5ms/step - loss: 0.8774 - accuracy: 0.6022 - val_loss: 0.8693 - val_accuracy: 0.6042
Epoch 4/20
216/216 [==============================] - 1s 5ms/step - loss: 0.8584 - accuracy: 0.6129 - val_loss: 0.8392 - val_accuracy: 0.6267
Epoch 5/20
216/216 [==============================] - 1s 5ms/step - loss: 0.8523 - accuracy: 0.6129 - val_loss: 0.8111 - val_accuracy: 0.6372
Epoch 6/20
216/216 [==============================] - 1s 5ms/step - loss: 0.8452 - accuracy: 0.6252 - val_loss: 0.8397 - val_accuracy: 0.6128
Epoch 7/20
216/216 [==============================] - 1s 6ms/step - loss: 0.8559 - accuracy: 0.6175 - val_loss: 0.8215 - val_accuracy: 0.6366
Epoch 8/20
54/54 [==============================] - 0s 2ms/step - loss: 0.8259 - accuracy: 0.6372
Fold 1 - Loss: 0.8258816599845886, Accuracy: 0.63715279102325

In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

# Load your data
df = pd.read_csv(r"D:\Github portfolio\Hemophilia\Hemophilia-Severity-Predictor\datasets\resampled_data_adasyn.csv")

# Separate features and labels
X = df.drop(columns=['Severity'])
y = df['Severity']

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert labels to one-hot encoding
y_one_hot = to_categorical(y_encoded)

# Split data into training and validation sets
split_ratio = 0.8
split_index = int(len(X_scaled) * split_ratio)

X_train = X_scaled[:split_index]
X_val = X_scaled[split_index:]
y_train = y_one_hot[:split_index]
y_val = y_one_hot[split_index:]

# Reshape for RNN
X_train_rnn = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_val_rnn = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))

# Define the RNN model
model = Sequential()
model.add(SimpleRNN(units=50, return_sequences=True, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])))
model.add(Dropout(0.5))
model.add(SimpleRNN(units=50, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))  # Assuming Severity has 3 classes: 0, 1, 2

# Compile the model
optimizer = Adam(learning_rate=0.001)  # You can experiment with different learning rates
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

# Train the model
history = model.fit(X_train_rnn, y_train, epochs=20, batch_size=32, validation_data=(X_val_rnn, y_val),
                    callbacks=[early_stopping, model_checkpoint], verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_val_rnn, y_val)
print(f'Loss: {loss}, Accuracy: {accuracy}')

# Clear the model from memory
K.clear_session()


Epoch 1/20
216/216 [==============================] - 3s 5ms/step - loss: 1.1016 - accuracy: 0.4770 - val_loss: 1.2466 - val_accuracy: 0.2500
Epoch 2/20
 48/216 [=====>........................] - ETA: 0s - loss: 0.9380 - accuracy: 0.5560

C:\Users\saray\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


216/216 [==============================] - 1s 5ms/step - loss: 0.9047 - accuracy: 0.5735 - val_loss: 1.2238 - val_accuracy: 0.2755
Epoch 3/20
216/216 [==============================] - 1s 5ms/step - loss: 0.8472 - accuracy: 0.6100 - val_loss: 1.1703 - val_accuracy: 0.3351
Epoch 4/20
216/216 [==============================] - 1s 5ms/step - loss: 0.8155 - accuracy: 0.6287 - val_loss: 1.1352 - val_accuracy: 0.4659
Epoch 5/20
216/216 [==============================] - 1s 4ms/step - loss: 0.7855 - accuracy: 0.6401 - val_loss: 1.1371 - val_accuracy: 0.3692
Epoch 6/20
216/216 [==============================] - 1s 5ms/step - loss: 0.7679 - accuracy: 0.6497 - val_loss: 1.1339 - val_accuracy: 0.3767
Epoch 7/20
216/216 [==============================] - 1s 6ms/step - loss: 0.7550 - accuracy: 0.6579 - val_loss: 1.0778 - val_accuracy: 0.4161
Epoch 8/20
216/216 [==============================] - 1s 6ms/step - loss: 0.7374 - accuracy: 0.6682 - val_loss: 1.0734 - val_accuracy: 0.5307
Epoch 9/20
216/21

In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load your data
df = pd.read_csv(r"D:\Github portfolio\Hemophilia\Hemophilia-Severity-Predictor\datasets\resampled_data_adasyn.csv")

# Separate features and labels
X = df.drop(columns=['Severity'])
y = df['Severity']

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert labels to one-hot encoding
y_one_hot = to_categorical(y_encoded)

# K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # Adjust n_splits as needed

fold_accuracies = []  # List to store accuracy of each fold

fold = 1
for train_index, val_index in kf.split(X_scaled):
    print(f"Processing fold {fold}")
    
    # Split data
    X_train, X_val = X_scaled[train_index], X_scaled[val_index]
    y_train, y_val = y_one_hot[train_index], y_one_hot[val_index]
    
    # Reshape for RNN
    X_train_rnn = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_val_rnn = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
    
    # Define the RNN model
    model = Sequential()
    model.add(SimpleRNN(units=100, return_sequences=True, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])))
    model.add(Dropout(0.5))
    model.add(SimpleRNN(units=50, return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))  # Assuming Severity has 3 classes: 0, 1, 2

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

    # Train the model
    history = model.fit(X_train_rnn, y_train, epochs=20, batch_size=32, validation_data=(X_val_rnn, y_val),
                        callbacks=[early_stopping, reduce_lr], verbose=1)
    
    # Evaluate the model
    loss, accuracy = model.evaluate(X_val_rnn, y_val)
    print(f'Fold {fold} - Loss: {loss}, Accuracy: {accuracy}')
    
    # Store the accuracy of this fold
    fold_accuracies.append(accuracy)
    
    # Increment fold counter
    fold += 1

    # Clear the model from memory
    K.clear_session()

# Calculate average accuracy
average_accuracy = np.mean(fold_accuracies)
print(f'Average Accuracy across all folds: {average_accuracy}')


Processing fold 1
Epoch 1/20
216/216 [==============================] - 4s 7ms/step - loss: 1.1734 - accuracy: 0.4748 - val_loss: 0.8751 - val_accuracy: 0.5718 - lr: 0.0010
Epoch 2/20
216/216 [==============================] - 1s 4ms/step - loss: 0.9666 - accuracy: 0.5492 - val_loss: 0.8304 - val_accuracy: 0.6007 - lr: 0.0010
Epoch 3/20
216/216 [==============================] - 1s 4ms/step - loss: 0.8744 - accuracy: 0.5932 - val_loss: 0.8204 - val_accuracy: 0.6065 - lr: 0.0010
Epoch 4/20
216/216 [==============================] - 1s 4ms/step - loss: 0.8349 - accuracy: 0.6206 - val_loss: 0.8054 - val_accuracy: 0.6175 - lr: 0.0010
Epoch 5/20
216/216 [==============================] - 1s 4ms/step - loss: 0.7967 - accuracy: 0.6401 - val_loss: 0.7973 - val_accuracy: 0.6337 - lr: 0.0010
Epoch 6/20
216/216 [==============================] - 1s 5ms/step - loss: 0.7817 - accuracy: 0.6510 - val_loss: 0.7882 - val_accuracy: 0.6441 - lr: 0.0010
Epoch 7/20
216/216 [==============================] 

In [9]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load your data
df = pd.read_csv(r"D:\Github portfolio\Hemophilia\Hemophilia-Severity-Predictor\datasets\resampled_data_adasyn.csv")

# Separate features and labels
X = df.drop(columns=['Severity'])
y = df['Severity']

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert labels to one-hot encoding
y_one_hot = to_categorical(y_encoded)

# K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # Adjust n_splits as needed

fold_accuracies = []  # List to store accuracy of each fold

fold = 1
for train_index, val_index in kf.split(X_scaled):
    print(f"Processing fold {fold}")
    
    # Split data
    X_train, X_val = X_scaled[train_index], X_scaled[val_index]
    y_train, y_val = y_one_hot[train_index], y_one_hot[val_index]
    
    # Reshape for RNN
    X_train_rnn = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_val_rnn = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
    
    # Define the RNN model
    model = Sequential()
    model.add(SimpleRNN(units=50, return_sequences=False, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])))
    model.add(Dense(3, activation='softmax'))  # Assuming Severity has 3 classes: 0, 1, 2

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

    # Train the model
    history = model.fit(X_train_rnn, y_train, epochs=20, batch_size=64, validation_data=(X_val_rnn, y_val),
                        callbacks=[early_stopping, reduce_lr], verbose=1)
    
    # Evaluate the model
    loss, accuracy = model.evaluate(X_val_rnn, y_val)
    print(f'Fold {fold} - Loss: {loss}, Accuracy: {accuracy}')
    
    # Store the accuracy of this fold
    fold_accuracies.append(accuracy)
    
    # Increment fold counter
    fold += 1

    # Clear the model from memory
    K.clear_session()

# Calculate average accuracy
average_accuracy = np.mean(fold_accuracies)
print(f'Average Accuracy across all folds: {average_accuracy}')


Processing fold 1
Epoch 1/20
108/108 [==============================] - 2s 9ms/step - loss: 0.9700 - accuracy: 0.5414 - val_loss: 0.8474 - val_accuracy: 0.6094 - lr: 0.0010
Epoch 2/20
108/108 [==============================] - 0s 4ms/step - loss: 0.7774 - accuracy: 0.6466 - val_loss: 0.8116 - val_accuracy: 0.6343 - lr: 0.0010
Epoch 3/20
108/108 [==============================] - 0s 4ms/step - loss: 0.7170 - accuracy: 0.6754 - val_loss: 0.7947 - val_accuracy: 0.6308 - lr: 0.0010
Epoch 4/20
108/108 [==============================] - 0s 4ms/step - loss: 0.6809 - accuracy: 0.6986 - val_loss: 0.7806 - val_accuracy: 0.6568 - lr: 0.0010
Epoch 5/20
108/108 [==============================] - 1s 5ms/step - loss: 0.6506 - accuracy: 0.7167 - val_loss: 0.7705 - val_accuracy: 0.6782 - lr: 0.0010
Epoch 6/20
108/108 [==============================] - 1s 5ms/step - loss: 0.6228 - accuracy: 0.7332 - val_loss: 0.7520 - val_accuracy: 0.6887 - lr: 0.0010
Epoch 7/20
108/108 [==============================] 

In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import RMSprop

# Load your data
df = pd.read_csv(r"D:\Github portfolio\Hemophilia\Hemophilia-Severity-Predictor\datasets\resampled_data_adasyn.csv")

# Separate features and labels
X = df.drop(columns=['Severity'])
y = df['Severity']

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert labels to one-hot encoding
y_one_hot = to_categorical(y_encoded)

# K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold_accuracies = []

fold = 1
for train_index, val_index in kf.split(X_scaled):
    print(f"Processing fold {fold}")
    
    # Split data
    X_train, X_val = X_scaled[train_index], X_scaled[val_index]
    y_train, y_val = y_one_hot[train_index], y_one_hot[val_index]
    
    # Reshape for RNN
    X_train_rnn = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_val_rnn = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
    
    # Define the RNN model
    model = Sequential()
    model.add(SimpleRNN(units=70, return_sequences=True, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])))
    model.add(SimpleRNN(units=30, return_sequences=False))
    model.add(Dense(3, activation='softmax'))

    # Compile the model
    optimizer = RMSprop(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

    # Train the model
    history = model.fit(X_train_rnn, y_train, epochs=30, batch_size=64, validation_data=(X_val_rnn, y_val),
                        callbacks=[early_stopping, reduce_lr], verbose=1)
    
    # Evaluate the model
    loss, accuracy = model.evaluate(X_val_rnn, y_val)
    print(f'Fold {fold} - Loss: {loss}, Accuracy: {accuracy}')
    
    # Store the accuracy of this fold
    fold_accuracies.append(accuracy)
    
    # Increment fold counter
    fold += 1

    # Clear the model from memory
    K.clear_session()

# Calculate average accuracy
average_accuracy = np.mean(fold_accuracies)
print(f'Average Accuracy across all folds: {average_accuracy}')


Processing fold 1
Epoch 1/30
108/108 [==============================] - 3s 8ms/step - loss: 0.9079 - accuracy: 0.5701 - val_loss: 0.8312 - val_accuracy: 0.5978 - lr: 0.0010
Epoch 2/30
108/108 [==============================] - 0s 4ms/step - loss: 0.7705 - accuracy: 0.6558 - val_loss: 0.8075 - val_accuracy: 0.6435 - lr: 0.0010
Epoch 3/30
108/108 [==============================] - 1s 6ms/step - loss: 0.7222 - accuracy: 0.6851 - val_loss: 0.7824 - val_accuracy: 0.6568 - lr: 0.0010
Epoch 4/30
108/108 [==============================] - 1s 6ms/step - loss: 0.6860 - accuracy: 0.7030 - val_loss: 0.7819 - val_accuracy: 0.6568 - lr: 0.0010
Epoch 5/30
108/108 [==============================] - 1s 6ms/step - loss: 0.6578 - accuracy: 0.7203 - val_loss: 0.7665 - val_accuracy: 0.6730 - lr: 0.0010
Epoch 6/30
108/108 [==============================] - 0s 4ms/step - loss: 0.6386 - accuracy: 0.7254 - val_loss: 0.7522 - val_accuracy: 0.6806 - lr: 0.0010
Epoch 7/30
108/108 [==============================] 

In [11]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

# Load your data
df = pd.read_csv(r"D:\Github portfolio\Hemophilia\Hemophilia-Severity-Predictor\datasets\resampled_data_adasyn.csv")

# Separate features and labels
X = df.drop(columns=['Severity'])
y = df['Severity']

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert labels to one-hot encoding
y_one_hot = to_categorical(y_encoded)

# Stratified K-Fold Cross-Validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fold_accuracies = []

fold = 1
for train_index, val_index in skf.split(X_scaled, y_encoded):
    print(f"Processing fold {fold}")
    
    # Split data
    X_train, X_val = X_scaled[train_index], X_scaled[val_index]
    y_train, y_val = y_one_hot[train_index], y_one_hot[val_index]
    
    # Reshape for RNN
    X_train_rnn = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_val_rnn = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
    
    # Define the RNN model
    model = Sequential()
    model.add(SimpleRNN(units=50, return_sequences=True, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.3))
    model.add(SimpleRNN(units=25, return_sequences=False, kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.3))
    model.add(Dense(3, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

    # Train the model
    history = model.fit(X_train_rnn, y_train, epochs=30, batch_size=64, validation_data=(X_val_rnn, y_val),
                        callbacks=[early_stopping, reduce_lr], verbose=1)
    
    # Evaluate the model
    loss, accuracy = model.evaluate(X_val_rnn, y_val)
    print(f'Fold {fold} - Loss: {loss}, Accuracy: {accuracy}')
    
    # Store the accuracy of this fold
    fold_accuracies.append(accuracy)
    
    # Increment fold counter
    fold += 1

    # Clear the model from memory
    K.clear_session()

# Calculate average accuracy
average_accuracy = np.mean(fold_accuracies)
print(f'Average Accuracy across all folds: {average_accuracy}')


Processing fold 1
Epoch 1/30
108/108 [==============================] - 3s 9ms/step - loss: 1.8378 - accuracy: 0.4813 - val_loss: 1.4231 - val_accuracy: 0.5862 - lr: 0.0010
Epoch 2/30
108/108 [==============================] - 1s 5ms/step - loss: 1.3290 - accuracy: 0.5906 - val_loss: 1.2087 - val_accuracy: 0.6209 - lr: 0.0010
Epoch 3/30
108/108 [==============================] - 1s 5ms/step - loss: 1.1495 - accuracy: 0.6223 - val_loss: 1.0946 - val_accuracy: 0.6221 - lr: 0.0010
Epoch 4/30
108/108 [==============================] - 1s 5ms/step - loss: 1.0477 - accuracy: 0.6325 - val_loss: 1.0388 - val_accuracy: 0.6047 - lr: 0.0010
Epoch 5/30
108/108 [==============================] - 0s 5ms/step - loss: 0.9824 - accuracy: 0.6336 - val_loss: 0.9827 - val_accuracy: 0.6250 - lr: 0.0010
Epoch 6/30
108/108 [==============================] - 1s 5ms/step - loss: 0.9463 - accuracy: 0.6327 - val_loss: 0.9597 - val_accuracy: 0.6181 - lr: 0.0010
Epoch 7/30
108/108 [==============================] 

In [12]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load your data
df = pd.read_csv(r"D:\Github portfolio\Hemophilia\Hemophilia-Severity-Predictor\datasets\resampled_data_adasyn.csv")

# Separate features and labels
X = df.drop(columns=['Severity'])
y = df['Severity']

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert labels to one-hot encoding
y_one_hot = to_categorical(y_encoded)

# Stratified K-Fold Cross-Validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fold_accuracies = []

fold = 1
for train_index, val_index in skf.split(X_scaled, y_encoded):
    print(f"Processing fold {fold}")
    
    # Split data
    X_train, X_val = X_scaled[train_index], X_scaled[val_index]
    y_train, y_val = y_one_hot[train_index], y_one_hot[val_index]
    
    # Reshape for RNN
    X_train_rnn = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_val_rnn = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
    
    # Define the RNN model
    model = Sequential()
    model.add(SimpleRNN(units=50, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), return_sequences=False))
    model.add(Dense(3, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

    # Train the model
    history = model.fit(X_train_rnn, y_train, epochs=20, batch_size=64, validation_data=(X_val_rnn, y_val),
                        callbacks=[early_stopping, reduce_lr], verbose=1)
    
    # Evaluate the model
    loss, accuracy = model.evaluate(X_val_rnn, y_val)
    print(f'Fold {fold} - Loss: {loss}, Accuracy: {accuracy}')
    
    # Store the accuracy of this fold
    fold_accuracies.append(accuracy)
    
    # Increment fold counter
    fold += 1

    # Clear the model from memory
    K.clear_session()

# Calculate average accuracy
average_accuracy = np.mean(fold_accuracies)
print(f'Average Accuracy across all folds: {average_accuracy}')


Processing fold 1
Epoch 1/20
108/108 [==============================] - 2s 7ms/step - loss: 0.9853 - accuracy: 0.5401 - val_loss: 0.8610 - val_accuracy: 0.5943 - lr: 0.0010
Epoch 2/20
108/108 [==============================] - 0s 4ms/step - loss: 0.7810 - accuracy: 0.6479 - val_loss: 0.8090 - val_accuracy: 0.6296 - lr: 0.0010
Epoch 3/20
108/108 [==============================] - 0s 4ms/step - loss: 0.7195 - accuracy: 0.6796 - val_loss: 0.7909 - val_accuracy: 0.6487 - lr: 0.0010
Epoch 4/20
108/108 [==============================] - 0s 4ms/step - loss: 0.6839 - accuracy: 0.6966 - val_loss: 0.7786 - val_accuracy: 0.6464 - lr: 0.0010
Epoch 5/20
108/108 [==============================] - 1s 5ms/step - loss: 0.6560 - accuracy: 0.7167 - val_loss: 0.7776 - val_accuracy: 0.6690 - lr: 0.0010
Epoch 6/20
108/108 [==============================] - 1s 5ms/step - loss: 0.6309 - accuracy: 0.7332 - val_loss: 0.7619 - val_accuracy: 0.6817 - lr: 0.0010
Epoch 7/20
108/108 [==============================] 